In [1]:
from qdrant_client import QdrantClient
from qdrant_client.http import models
from qdrant_client.http.models import CollectionStatus

In [2]:
client = QdrantClient(host="localhost", port=6333)
client

In [3]:
my_collection = "first_collection"

first_collection = client.recreate_collection(
    collection_name=my_collection,
    vectors_config=models.VectorParams(size=100, distance=models.Distance.COSINE)
)
print(first_collection)

True


In [4]:
collection_info = client.get_collection(collection_name=my_collection)
list(collection_info)

[('status', <CollectionStatus.GREEN: 'green'>),
 ('optimizer_status', <OptimizersStatusOneOf.OK: 'ok'>),
 ('vectors_count', 0),
 ('indexed_vectors_count', 0),
 ('points_count', 0),
 ('segments_count', 8),
 ('config',
  CollectionConfig(params=CollectionParams(vectors=VectorParams(size=100, distance=<Distance.COSINE: 'Cosine'>, hnsw_config=None, quantization_config=None, on_disk=None), shard_number=1, sharding_method=None, replication_factor=1, write_consistency_factor=1, read_fan_out_factor=None, on_disk_payload=True, sparse_vectors=None), hnsw_config=HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0, on_disk=False, payload_m=None), optimizer_config=OptimizersConfig(deleted_threshold=0.2, vacuum_min_vector_number=1000, default_segment_number=0, max_segment_size=None, memmap_threshold=None, indexing_threshold=20000, flush_interval_sec=5, max_optimization_threads=None), wal_config=WalConfig(wal_capacity_mb=32, wal_segments_ahead=0), quantization_config=

In [5]:
import numpy as np

In [6]:
data = np.random.uniform(low=-1.0, high=1.0, size=(1_000, 100))
type(data[0, 0]), data[:2, :20]

(numpy.float64,
 array([[ 0.84439167, -0.19918004, -0.61638736,  0.72398117,  0.4626335 ,
         -0.64837971,  0.39309187,  0.20166689, -0.11510416, -0.85854435,
         -0.10822716,  0.97322058,  0.1117175 , -0.85425979, -0.39769858,
          0.15855545,  0.05905023,  0.099604  ,  0.01759705, -0.51561161],
        [ 0.2583855 , -0.92291498, -0.52936709, -0.11128443, -0.85106515,
         -0.94130676,  0.42319266,  0.36355282,  0.53345633,  0.6313361 ,
         -0.79149132, -0.15043201,  0.469217  , -0.18505916, -0.90230898,
         -0.81998317,  0.60496644,  0.78355021,  0.8703824 ,  0.40341385]]))

In [7]:
index = list(range(len(data)))
index[-10:]

[990, 991, 992, 993, 994, 995, 996, 997, 998, 999]

In [8]:
client.upsert(
    collection_name=my_collection,
    points=models.Batch(
        ids=index,
        vectors=data.tolist()
    )
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [9]:
client.retrieve(
    collection_name=my_collection,
    ids=[100],
    with_vectors=True # the default is False
)

[Record(id=100, payload={}, vector=[0.04586303, -0.12236376, 0.07361238, 0.1290181, 0.081923746, 0.016158417, -0.14562848, 0.08458243, -0.031297635, -0.13974294, -0.08105281, 0.09390469, 0.061711494, -0.06162026, 0.16921076, 0.005850575, -0.18021618, 0.046764027, -0.15072034, -0.14405674, 0.01980764, -0.17589813, -0.16778044, -0.042625286, 0.087717816, 0.1749383, -0.060236596, 0.04281515, -0.021478314, 0.11451723, -0.026612358, 0.14152445, 0.12004151, -0.13624181, 0.113061935, -0.029496973, 0.10543082, -0.07494068, -0.012730644, 0.11382175, 0.17174783, 0.053966332, -0.06885202, 0.13197796, -0.10459465, 0.0038037582, -0.14315183, -0.036238264, -0.022871453, 0.1398432, 0.0307088, -0.1404576, 0.1012566, -0.09838463, -0.03314703, -0.028993018, -0.021312948, 0.05362477, 0.17641188, -0.0318828, 0.11421832, -0.10294863, -0.026903123, -0.042825118, -0.13673201, 0.08164635, 0.15495121, -0.009972439, -0.0918386, -0.09405637, 0.11648503, -0.079037204, -0.16316728, 0.0422289, 0.088015586, -0.02113

In [10]:
def create_song():
    return np.random.uniform(low=-1.0, high=1.0, size=100).tolist()

In [11]:
client.upsert(
    collection_name=my_collection,
    points=[
        models.PointStruct(
            id=1000,
            vector=create_song(),
        )
    ]
)

UpdateResult(operation_id=1, status=<UpdateStatus.COMPLETED: 'completed'>)

In [13]:
client.count(
    collection_name=my_collection, 
    exact=True,
)

CountResult(count=1001)

In [14]:
client.delete(
    collection_name=my_collection,
    points_selector=models.PointIdsList(
        points=[1000],
    ),
)

UpdateResult(operation_id=2, status=<UpdateStatus.COMPLETED: 'completed'>)

In [15]:
from faker import Faker

In [16]:
fake_something = Faker()
fake_something.name()

'Angel Doyle'

In [17]:
payload = []

for i in range(len(data)):
    payload.append(
        {
            "artist":   fake_something.name(),
            "song":     " ".join(fake_something.words()),
            "url_song": fake_something.url(),
            "year":     fake_something.year(),
            "country":  fake_something.country()
        }
    )

In [18]:
client.upsert(
    collection_name=my_collection,
    points=models.Batch(
        ids=index,
        vectors=data.tolist(),
        payloads=payload
    )
)

UpdateResult(operation_id=3, status=<UpdateStatus.COMPLETED: 'completed'>)

In [19]:
resutls = client.retrieve(
    collection_name=my_collection,
    ids=[10, 50, 100, 500],
    with_vectors=False
)

type(resutls), resutls

(list,
 [Record(id=10, payload={'artist': 'Preston Price', 'country': 'Nigeria', 'song': 'upon yet management', 'url_song': 'http://www.lopez.com/', 'year': '2009'}, vector=None, shard_key=None),
  Record(id=50, payload={'artist': 'Victoria Clark', 'country': 'Sudan', 'song': 'check growth there', 'url_song': 'http://lane-hayes.com/', 'year': '2021'}, vector=None, shard_key=None),
  Record(id=100, payload={'artist': 'Henry Macdonald', 'country': 'Libyan Arab Jamahiriya', 'song': 'doctor lawyer rate', 'url_song': 'https://www.harris-payne.com/', 'year': '2003'}, vector=None, shard_key=None),
  Record(id=500, payload={'artist': 'Lori Stephenson', 'country': 'Northern Mariana Islands', 'song': 'product but evening', 'url_song': 'http://www.donovan.com/', 'year': '1983'}, vector=None, shard_key=None)])

In [20]:
living_la_vida_loca = create_song()

In [21]:
client.search(
    collection_name=my_collection,
    query_vector=living_la_vida_loca,
    limit=3
)

[ScoredPoint(id=918, version=3, score=0.33943692, payload={'artist': 'Michael Rivas', 'country': 'Jersey', 'song': 'across resource only', 'url_song': 'https://delgado.org/', 'year': '1990'}, vector=None, shard_key=None),
 ScoredPoint(id=970, version=3, score=0.29493442, payload={'artist': 'Gina Martinez', 'country': 'Libyan Arab Jamahiriya', 'song': 'investment number with', 'url_song': 'http://www.lamb.com/', 'year': '1982'}, vector=None, shard_key=None),
 ScoredPoint(id=612, version=3, score=0.2500589, payload={'artist': 'Michele Gutierrez', 'country': 'Guam', 'song': 'dinner guy success', 'url_song': 'http://www.turner-brown.com/', 'year': '1981'}, vector=None, shard_key=None)]

In [22]:
aussie_songs = models.Filter(
    must=[models.FieldCondition(key="country", match=models.MatchValue(value="Australia"))]
)
type(aussie_songs)

qdrant_client.http.models.models.Filter

In [23]:
client.search(
    collection_name=my_collection,
    query_vector=living_la_vida_loca,
    query_filter=aussie_songs,
    limit=2
)

[ScoredPoint(id=439, version=3, score=0.08562013, payload={'artist': 'Amanda Chase', 'country': 'Australia', 'song': 'executive that vote', 'url_song': 'https://www.bridges.org/', 'year': '1971'}, vector=None, shard_key=None),
 ScoredPoint(id=873, version=3, score=0.05005426, payload={'artist': 'William Hall', 'country': 'Australia', 'song': 'team owner design', 'url_song': 'https://www.guerrero.net/', 'year': '2018'}, vector=None, shard_key=None)]

In [24]:
client.search(
    collection_name=my_collection,
    query_vector=living_la_vida_loca,
    query_filter=aussie_songs,
    with_payload=models.PayloadSelectorExclude(exclude=["year"]),
    limit=5
)

[ScoredPoint(id=439, version=3, score=0.08562013, payload={'artist': 'Amanda Chase', 'country': 'Australia', 'song': 'executive that vote', 'url_song': 'https://www.bridges.org/'}, vector=None, shard_key=None),
 ScoredPoint(id=873, version=3, score=0.05005426, payload={'artist': 'William Hall', 'country': 'Australia', 'song': 'team owner design', 'url_song': 'https://www.guerrero.net/'}, vector=None, shard_key=None),
 ScoredPoint(id=528, version=3, score=-0.03573903, payload={'artist': 'Dennis Gillespie', 'country': 'Australia', 'song': 'difficult during moment', 'url_song': 'https://carpenter-johnson.com/'}, vector=None, shard_key=None)]

In [25]:
client.clear_payload(
    collection_name=my_collection,
    points_selector=models.PointIdsList(
        points=index,
    )
)

UpdateResult(operation_id=4, status=<UpdateStatus.COMPLETED: 'completed'>)